## Scipy and Optimization

In the last notebook we took a tour of the `numpy` and `scipy` stack as it relates to math and statistics. This notebook is going to focus on another subset of `scipy` functionality that is absolutely critical to data analytics. Nearly every statistical algorithm aside from Ordinary Least Squares (OLS) regression cannot be solved algebraically. Because they cannot be solved algebraically, they must be solved **numerically**.

### Numeric Optimization

Let's start with an example of optimization. Let's say that you know the demand function for tickets to watch a local sports franchise. You can write the inverse demand function as

$$ P = 300 - \frac{1}{2}Q $$

and the total cost function as

$$ TC = 4000 + 45Q $$

In order to choose the right number of tickets to sell, you need to calculate the quantity of tickets that will maximize profits. We can calculate total revenue as $ TR = P \times Q $, and we can calculate profit as $ \Pi = TR - TC $. This means that our profit function is

$$ \Pi = 300Q - \frac{1}{2}Q^2 - 4000 - 45Q $$

In order to find the $Q$ associated with the highest achievable level of profit, we can use calculus to find the point at which the rate of change in the profit function is zero ($\frac{\partial\Pi}{\partial Q}=0$).

$$ \frac{\partial\Pi}{\partial Q} = 300 - Q - 45 = 0 \implies Q = 255$$

So we can **algebraically** solve this particular problem. This isn't always the case. Using `scipy`, we can solve this same problem, as well as many algebraically intractable problems that might be more interesting to us.


<img src="https://github.com/dustywhite7/Econ8320/raw/master/SlidesCode/paraboloid.png" width="200" height="200" />


In any optimization problem, we need to find a way to get ourselves to the minimum, and to know when we get there. When we look at the above image, we are able to visually trace the functional shape (looks like a rainbow ice cream cone to me...) and locate the bottom of the function. What we want to do is utilize an algorithm to "trace" our way from an arbitrary starting point within a function to the optimal point in that function.

In three or fewer dimensions, this is easy. Regressions and statistical models often live in worlds with 100's or 1000's (even millions sometimes) of dimensions. We can't visualize our way to the bottom of those functions!

The class of algorithm that is used to solve these problems is called **gradient descent**.

<img src="https://github.com/dustywhite7/Econ8320/raw/master/SlidesCode/gradDesc.png" width="400" />

**Gradient descent** is an algorithm that explores the shape of the function, and determines which direction is most likely to lead to the optimal point. Let's focus on minimization. We want to find our way to the *bottom* of a function, and we can use gradient descent to try to get there. Given any starting point, our goal is to check the direction in which we can move downward most quickly, and start moving in that direction. At some distance from our starting point, we will stop and re-evaluate the direction in which we would like to travel. Are we still heading downhill in the steepest direction? If we aren't, then we need to update our behavior.

Our gradient descent algorithm will keep "looking around" and moving down until it reaches a point at which it can no longer move "down" in any meaningful way. That is the stopping point, and is treated as the optimum.


With an intuitive understanding of how optimization will happen computationally, it's time to learn a bit more about the math and the code that will help us to achieve computational optimization.

Consider a function, $f$, with two variables $x$ and $y$. Because there are two input variables in the function, it has two partial derivatives:

$$ \frac{\partial f}{\partial x} \text{ and } \frac{\partial f}{\partial y} $$

Each partial derivative tells us how $f$ changes as we move in a particular dimension **all else equal**. The **gradient**, then, is the vector of all partial derivatives of a given function at any point along the function:

$$ \nabla f = \left[ \begin{matrix} \frac{\partial f}{\partial x} \\ \\ \frac{\partial f}{\partial y} \end{matrix} \right]  $$

We can use the gradient to determine the linear approximation of a function at any given point. Think about the gradient as the mathematical representation of the slope and direction of a hill you are hiking on. If we know the gradient, we know which way is down. As we continue to calculate gradients while walking, we can continue to ensure that we will walk downhill until we reach the bottom of the hill.




The steps of our gradient descent function will be the following:

- Evaluate the gradient of the function
- Find the direction of steepest descent
- Determine how far to move in that direction
- Move to new point
- Reevaluate the gradient
- Stop moving when gradient is within a margin of error from 0

Let's try to implement gradient descent by solving our old profit maximization problem computationally. The very first thing that we need to do is write a Python function that will represent our mathematical function.

In [ ]:
def profit(q):
    p = 300-0.5*q
    tr = p*q
    tc = 4000 + 45*q
    return tr - tc

This function will allow us to calculate profit at any output level based on our assumed total costs and demand curve. With this function, we can quickly calculate the gradient (in this case, just a simple derivative because our function is univariate) by calculating profit at two nearby points, and dividing by the horizontal distance between those points:

In [ ]:
# Gradient at q=200

(profit(201) - profit(199))/2

    55.0



Thus, a one unit increase in output at $Q=200$ results in a $55 increase in profits. This is cool, but it isn't enough for us to find the point of maximum profit (the optimal point). For that, we will need to calculate LOTS of gradients in order to move along the function until we cannot increase profits any further.

Fortunately for us, `scipy` comes with optimization tools that will do all of the heavy lifting of the "search" for the optimal point. All that we need to do is frame our question algorithmically, and let `scipy` do the rest:

In [ ]:
import numpy as np
from scipy.optimize import minimize

We start by importing the `minimize` function from `scipy.optimize`. Hang on! Weren't we working on a MAXIMIZATION problem?? What are we doing here?

Maximization and minimization are the **same thing**. To maximize a function, you can multiply that function by `-1` and then calculate the minimum of the new "upside-down" function. It is functionally equivalent. So, in computational optimization, we always minimize.

### Prepping for optimization

As we prepare to optimize, there are two common problems with our function that we may need to resolve:

1) When using `minimize` we can only pass an array of inputs, so we have to be careful to write our function accordingly
2) Our problem is concave, and so has a maximum
	- We need to restate it as a minimization problem

Problem 1 does not apply, since our function in univariate. In order to make our problem a minimization problem, we can flip our profit maximization function. We will simply return -1 * profit:

In [ ]:
def profit(q):
    p = 300-0.5*q
    tr = p*q
    tc = 4000 + 45*q
    pi =  tr - tc # didn't name it profit because that is our function's name! Don't want to clutter our name space!
    return -1*pi

### Making the call to `minimize`

Now that our function is ready, it is time to minimize! The `minimize` function takes two arguments:
1. Our function that we want to optimize
2. A starting guess (as a vector)

Let's give it a try.

In [ ]:
res = minimize(profit, [0]) # provide function and starting inputs
res

          fun: -28512.499999980355
     hess_inv: array([[1.00000175]])
          jac: array([0.])
      message: 'Optimization terminated successfully.'
         nfev: 21
          nit: 3
         njev: 7
       status: 0
      success: True
            x: array([255.00019821])



That's it! No calculus, no searching, no checking gradients manually. `minimize` simply takes our function and our starting guess and brings us back the optimal choice. We get lots of information stored in the attributes of the `res` object:

- `fun` provides the value of the function (this is -1 times the profit level at the optimal output in our example)
- `hess_inv` and `jac` are measures of gradient and are used to determine how far to go and in which direction
- `message` should be self-explanatory
- `nfev` is the number of times the function (in this case `profit`) was evaluated during the search
- `nit` is the number of iterations it took to find the optimum
- `njev` is the number of times the Jacobian was estimated
- `status` is a code associated with the `message` and `success` atrributes
- `success` tells you whether or not an optimum was found (sometimes it cannot be easily found!)
- `x` probably the most important attribute. This tells us the optimal input value (in our case $Q$), or set of values depending on our function. In a regression context, this could represent the fitted coefficients!

Going forward, you will realize (especially because so many of them print the output as they optimize) just how many libraries in Python use this minimize function behind the scenes. It is used in `statsmodels`, `sklearn`, and many other high-profile libraries! Now that you know where it really happens (in `scipy`!), you'll be better able to troubleshoot the problems that will inevitably arise as you use statistical models.

**Solve-it!**

In this lesson we learned about optimization using SciPy. For the assignment this week, I would like you to build off of your `RegressionModel` class. You will add a Logistic Regression (Logit) method to your class, so that when the `regression_type` parameter is `logit`, Logistic Regression Results are returned.

Your job is to create the following functionality within your class object:
- a method (call it `logistic_regression`) that estimates the results of logistic regression using your `x` and `y` data frames, and using a likelihood function and gradient descent (DO NOT USE PREBUILT REGRESSION FUNCTIONS).
    - You need to write a function to calculate the Log-likelihood of your model
    - You need to implement gradient descent to find the optimal values of beta
    - You need to use your beta estimates, the model variance, and calculate the standard errors of the coefficients, as well as Z statistics and p-values
    - the results should be stored in a dictionary named `results`, where each variable name (including the intercept if `create_intercept` is `True`) is the key, and the value is another dictionary, with keys for `coefficient`, `standard_error`, `z_stat`, and `p_value`. The coefficient should be the log odds-ratio (which takes the place of the coefficients in OLS)
- a method called `fit_model` that uses the `self.regression_type` attribute to determine whether or not to run an OLS or Logistic Regression using the data provided. This method should call the correct regression method.
- an updated method (call it `summary`) that presents your regression results in a table
    - Columns should be: Variable name, Log odds-ratio value, standard error, z-statistic, and p-value, in that order.
    - Your summary table should have different column titles for OLS and Logistic Regressions! (think if statement...)

You only need to define the class. My code will create an instance of your class (be sure all the names match these instructions, and those from last week!), and provide data to run a regression. I will provide the same data to you, so that you can experiment and make sure that your code is functioning properly.

NOTE: I have created a [primer on Logistic regression](https://github.com/dustywhite7/Econ8320/blob/master/SlidesPDF/9-2%20-%20Logit%20Primer.pdf) to go with this assignment. See the Github slidesPDF folder

Put the code that you would like graded in the cell labeled `#si-exercise`. I recommend copying your code from the last assignment (in chapter 9 about linear regression and `numpy`), and continuing from there.



In [60]:
#si-exercise

#regression model from last class
import pandas as pd
import numpy as np
from scipy.stats import t

class RegressionModel:
    def __init__(self, x, y, create_intercept=True, regression_type='ols'):
        self.x = x.copy()
        self.y = y
        self.create_intercept = create_intercept
        self.regression_type = regression_type
        self.results = {}
        if self.create_intercept:
            self.add_intercept()

    def add_intercept(self):

        self.x['intercept'] = 1

    def ols_regression(self):

        X = self.x.values
        Y = self.y.values.reshape(-1, 1)

        XtX_inv = np.linalg.inv(np.dot(X.T, X))
        XtY = np.dot(X.T, Y)
        coefficients = np.dot(XtX_inv, XtY)

        predictions = np.dot(X, coefficients)
        residuals = Y - predictions

        n, k = X.shape
        s_square = (residuals.T @ residuals) / (n - k)
        variance = s_square[0][0]
        standard_errors = np.sqrt(np.diag(variance * XtX_inv))


        t_stats = coefficients.flatten() / standard_errors
        p_values = (1 - t.cdf((t_stats), df=n - k))


        for i, col in enumerate(self.x.columns):
            self.results[col] = {
                'coefficient': coefficients[i][0],
                'standard_error': standard_errors[i],
                't_stat': t_stats[i],
                'p_value': p_values[i]
            }
    def logistic_regression(self):
      def loglike(b,y,x):
        top = np.exp(np.dot(x,b))
        bottom = 1  + top
        gamma = top/bottom
        loglike = -1*np.sum(y*np.log(gamma) + (1-y)*np.log(1-gamma))
        return loglike

      #def grad_loglike(b,y,x):  #something about the derrivative, im getting stuck here ughgbghbkmf,d
       # top = np.exp(np.dot(x,b))
        #bottom = 1  + top
        #gamma = top/bottom
       # grad

       #retry, what other classmate said helped
      #def grad_loglike(b):  #something about the derrivative, im getting stuck here ughgbghbkmf,d
       # top = np.dot(x, b)
       # X_t_b = 1/ #x times beta

        def grad_loglike(beta):
          linear_combination = np.dot(self.x.values, beta)
          predicitions = 1/ 1(1 + np.exp(-linear_combination))
          return -np.dot(self.x.T.values, self.y - predictions)

        initial_beta = np.zeroes(self.x.shape[1])
        result = minimize(loglike, initial_beta, args=(self.y, self.x))

        if not result.success:
          raise RuntimeError("Optimization Failed")

        coefficient = result.x
        self.results

        predictions = 1/(1 + np.exp(-np.dot(self.x.values, beta_hat)))

        n,k = self.x.shape
        hessian_matrix = np.zeroes((k,k))

        for i in range(n):
          predi_i = predictions[i]
          hessian_matrix += predi_i * (1 - predi_i) * np.outer(self.x.iloc[i].values, self.x.iloc[i].values)

        hessian_matrix += np.eye(k) * 1e6

        try:
          inverse_hessian = np.linalg.inv(hessian_matrix)
          standard_error = np.sqrt(np.diag(inverse_hessian))
        except np.linalg.LinAlgError:

          standard_error = np.full(k, np.na )

          z_stat = beta_hat / standard_error
          p_value = 1 - norm.cdf(np.abs(z_stat))      #2 * (1 - t.cdf(np.abs(t_stats), df=n - k))


        for i, col in enumerate(self.x.columns):
            self.results[col] = {
                'coefficient': coefficient[i][0],
                'standard_error': standard_error[i],
                'z_stat': z_statz[i],
                'p_value': p_value[i]
            }

    def fit_model(self):
        if self.regression_type == 'ols':
            self.ols_regression()
        elif self.regression_type == 'logit':
            self.logistic_regression()
        else:
            raise ValueError("Invalid regression type")

    def summary(self):
        self.ols_regression()
        if self.regression_type == 'ols':
           summary_df = pd.DataFrame({
            "Variable name": list(self.results.keys()),
            "coefficient value": [self.results[var]['coefficient'] for var in self.results],
            "standard error": [self.results[var]['standard_error'] for var in self.results],
            "t-statistic": [self.results[var]['t_stat'] for var in self.results],
            "p-value": [self.results[var]['p_value'] for var in self.results]
        })
        elif self.regression_type == 'logit':
           summary_df = pd.DataFrame({
            "Variable name": list(self.results.keys()),
            "coefficient value": [self.results[var]['coefficient'] for var in self.results],
            "standard error": [self.results[var]['standard_error'] for var in self.results],
            "t-statistic": [self.results[var]['t_stat'] for var in self.results],
            "p-value": [self.results[var]['p_value'] for var in self.results]
        })

        print(summary_df)
        return summary_df

#end of regression model from last class



<>:72: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:72: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<ipython-input-60-ad3411e4c4b2>:72: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  predicitions = 1/ 1(1 + np.exp(-linear_combination))


In [70]:
#testing code
data = pd.read_csv("/content/assignment8Data.csv")
x = data.loc[:100, ['sex','age','educ']]
y = data.loc[:100, 'white']
reg = RegressionModel(x, y, create_intercept=True, regression_type='logit')
reg.fit_model()

sex = {'coefficient': -1.1229156890097627,
      'standard_error': 0.39798772782618025,
      'z_stat': -2.821483202869492,
      'p_value': 0.004780214077269219}
age = {'coefficient': -0.007012518056833769,
      'standard_error': 0.010835821823286998,
      'z_stat': -0.6471607019011091,
      'p_value': 0.5175279421902776}
educ = {'coefficient': -0.046485475816343394,
      'standard_error': 0.10100278092776117,
      'z_stat': -0.46023956359766527,
      'p_value': 0.6453442758780246}
intercept = {'coefficient': 5.735435005488546,
      'standard_error': 1.1266207023561843,
      'z_stat': 5.090830475148922,
      'p_value': 3.56498650369634e-07}

sexEq = (round(sex['coefficient']-reg.results['sex']['coefficient'], 1)==0) & (round(sex['standard_error']-reg.results['sex']['standard_error'], 1)==0) & (round(sex['z_stat']-reg.results['sex']['z_stat'], 1)==0) & (round(sex['p_value']-reg.results['sex']['p_value'], 1)==0)
ageEq = (round(age['coefficient']-reg.results['age']['coefficient'], 1)==0) &  (round(age['standard_error']-reg.results['age']['standard_error'], 1)==0) & (round(age['z_stat']-reg.results['age']['z_stat'], 1)==0) & (round(age['p_value']-reg.results['age']['p_value'], 1)==0)


KeyError: 'sex'

Other stuff for reference below:

In [6]:
#class video example for lesson 10


#estimate a gradient of the following function
#y = sin(x) - (z+3)**@
import numpy as np

def y(x, z):
  return np.sin(x) - (z+3)**2

y(1,10) #this gives us the value of y at x =1 and z =10

y(-3,2)

#make a gradient for the slope in each direction, find different points for different slopes
def grad_y(x,z):
  slope_x = (y(x + 0.05, z) - y(x,z))/0.05  #find the derrivative in thex direction (this is the slope in the x direction rise/run)
  slope_z = (y(x, z + 0.05) - y(x,z))/0.05 #find the derrivative in the z direction (this is the slope in the z direction rise/run)
  return (slope_x, slope_z)

grad_y(1,10) #shows the slope in the x and z position




(0.5190448157225092, -26.050000000000182)

In [8]:
#in class example continued

#minimization examples (there is no functon for maxamizing because it is the same as minimizing just mulitply by -1)
from scipy.optimize import minimize

#need to flip our function since its concave in order to turn it from max to min, minimize needs to be able to take a list of values( an array) snd not just 2 values. So our function just needs to be able to take one input of an array of xs

def q(xs):
  return -1*y(xs[0], xs[1])

res = minimize(q, [0,0]) #q is the function we want to pass and [0,0] is starting point
res

#this finds that our optomum is for x = 1.571 and z = -3

#plug these back into y function from above

y(1.571, -3) #what we get back from this is essentially 1

q([1.57, -3]) #is negative 1 because it is flipped of y




-0.9999996829318346

In [9]:
#try with different starting point at 10,10
minimize(q, [10,10])

#find that the optimum x changes (7.854) but the z stays the same(-3), this is because we have a sin function in the def y fucntion we defnied above. We have many minimum values wehre we have the same z but depending on where we start we can get a different optimal x
#because of this we cant randomly choose a starting point becuase this could change the optmum

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -0.9999999999992139
        x: [ 7.854e+00 -3.000e+00]
      nit: 6
      jac: [ 1.863e-07 -1.743e-06]
 hess_inv: [[ 1.000e+00 -1.399e-03]
            [-1.399e-03  5.070e-01]]
     nfev: 24
     njev: 8

In [10]:
#logistic regression (check this video for the function ill have to use for to solve this question on homework)
# output is now the probability of success -> p(y=1)

#any number raised to negative infinity is 0

sol = minimize(q, [0,0])
sol.x #<- the paramater attatched here are the beta coefficients for our model


array([ 1.57079632, -3.00000001])

OLS MLE FUNCTION EXAMPLE

In [13]:
#OLS MLE Function (building out likelihood function)

import numpy as np

#betas are k long
#sigma^2 is our variance and a single number
#pass in a single array and then extract the last value to be sigma^2

def ols_mle(theta, x, y): #theta is our general parameters (out inputs so our betas + our sigmas)
  beta = theta[:-1] #everything except for the last theta
  sig2 = theta[-1] #our last theta is going to be our sigma squared, all other thetas will be betas as defined above
  n = x.shape[0] #this is the number of rows in x

  #generate our likelihood
  first = -1 * (n/2) *np.log(2*np.pi) #first part of our likelihood function
  second = -1 * (n/2) *np.log(sig2)
  epsilon = y - (x @ beta) #this is our error #y, x, and beta are matrixes in this equation so we have to input them here with matrix algebra
  third = -1 * ((1/2*sig2)) * (epsilon.T @ epsilon) #epsilon.T trnasnposes

  return -1 * (first + second + third) #the negative one helps makes this minimzie (because orignially this function would find the max)




In [12]:
from scipy.optimize import minimize

minimize(ols_mle[0,0,0], #pass in a and y if not doing with a class) #if we had 2 x's we need 3 rwos because this will pass through x1, x2, and signma squared

TypeError: 'function' object is not subscriptable

END MLE FUNCTION EXAMPLE

OTHER STATS HELP From walkthrough code


In [19]:
#dot product
a = [1,2,3,4]
b = [10,20,31,3]
def dot_prod(a,b):
  dot = 0
  for i in range(len(a)):
    dot += a[i] * b[i]
  return dot

dot_prod(a,b)

#or we can do a second way

def dot_prod(a,b):
  return sum([i*j for i,j in zip(a,b)])

dot_prod(a,b)


#Matrix multiplication, we wont use but was just an example from videos

def mat_mul(A,B):
  matrix = []
  for i in range(len(A)):
    m_row = []
    for j in range(len(B[0])):
      row = A[i]
      col = [b_row[i] for b_row in B]
      m_row.append(dot_prod(row, col))
    matrix.append(m_row)
  return matrix


A = [[1,2],[3,4]]
B =[[6], [5]]

mat_mul(A,B)



IndentationError: expected an indented block after 'for' statement on line 45 (<ipython-input-19-6232cc7b2d20>, line 46)

In [24]:
#this is the code directl from the slides  for matrix multiplication that actual worked

def matMul(a, b): # Define function, take 2 matrices
  for i in range(len(a)): # Make sure a is matrix
    if len(a[i]) is not len(a[0]): # If not,
      raise RuntimeError( # Raise an error
        "Matrix A is not correctly specified")
  for i in range(len(b)): # Make sure b is a matrix
    if len(b[i]) is not len(b[0]): # If not,
      raise RuntimeError( # Raise an error
        "Matrix B is not correctly specified")
  matrix = [] # Initialize new empty matrix
  if len(a[0]) is len(b): # Test for conformability
    for i in range(len(a)): # Iterate over rows of a
      row = [] # Create row of new matrix
      for j in range(len(b[0])): # Iterate over columns
        row.append(dot_prod(a[i], # Append elements of col
          [b[n][j] for n in range(len(b))]))
        matrix.append(row) # Append rows to matrix
    return matrix # Return the newly calculated matrix
  else: # If matrices are nonconforming
    raise RuntimeError( #Raise an error
      "Matrices are nonconformable for multiplication")

A = [[1,2],[3,4]]
B =[[6], [5]]

matMul(A,B)



[[16], [38]]

In [44]:
from re import T
#using numpy instead for matrix multiplication

#core element of numpy is arrays
import numpy as np

A = [[1,2],[3,4]]
B =[[6], [5]]

np.array(A)

A = np.array(A)
np.shape(A) #tells me that a is a 2x2 matrix

A.T #this will give me the A matrix transposed

B = np.array(B)
np.shape(B)

A @ B #this @ will mutiply out matrixs'

np.zeros((3,3)) #gives us a 3x3 matrix with zeros in it, can create a tensor using (3,3,3)

np.eye(3) #gives us a 3x3 identity matrix

A.reshape(4,1) #this will reshape matrix a to be 4x1


A + 1 #this will add 1 to every element in A matrix (can do the same with subtraction and multiplication)

A.T.dot(A) #this will give us A**2 (A matrix * A matrix) had to transpose the first a first to get this before multplication using the dot function

A.T @ A #this will also let us square the matrix



#in class excersie

# y = a + b * x + c * x**2, try to create a function in these terms using matrix multiplication

def calc_y(a,b,c,x):
  mat1 = np.array([[a,b,c]]) #have to make sure everything is an array and not just a regular list with just []
  mat2 = np.array([[1, x, x**2]])
  y = mat1 @ mat2.T
  return y

calc_y(1,1,1,1)

array([[3]])

In [52]:
#random numbers
np.random.rand()
np.random.rand(10,1) #give be a 10x1 array of random numbers

#example: generate a sample of 10 obs from Exponenetial Distribution, where lambda = 1 (Look up CDF of exponential Distribution, and solve for x)


p = 1 - exp(-1 * x) #for cdf lambda = 1 so the values in teh parenthesis just become -x

#if were drawing p we need to invert this so that we can calculate x

exp (-1 * x) = 1-p
#get rid of exponentiation by taking the log
-x = np.log(1-p)
#now invert this
x = -1 * np.log(1-p)




SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (<ipython-input-52-ddfc4f35556c>, line 12)

In [54]:
#make a function using what we learned above, this will give us values from the exponential distribution
def exp():
  return -1 * np.log(1-np.random.rand())

exp()



array([0.17229774, 0.54666556, 1.75785322, 1.14429244, 0.68458634,
       0.18428722, 0.95002375, 0.69869035, 1.17329203, 0.13358234])

In [ ]:
#get it where with n argument ir will give us n numbers from the exponential distribtion
def exp(n):
  return -1 * np.log(1-np.random.rand(n))

exp(10)

In [56]:
#distribututional calculations
import scipy.stats as ss

ss.norm.sf(2) #will give us probability of a value being above the z score (in this case we did z score 2) will be (2.27%)

#allows us to do critical value and waht not

# to see probabilty below we use cdf
ss.norm.cdf(3) #shows at z score of the 3 the probabilty of a value being below z score (3) is .998 or 99.8%


0.9986501019683699